# 第二章：工具库入门 — NumPy、Matplotlib、ASE

本章面向**无编程基础**的读者，通过项目中的**真实数据**介绍三个核心库。  
如果你已熟悉这些库，可以跳到第三章。

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data_example" / "potential"
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

print("路径设置完成")

---

## 2.1 NumPy 数组

### 什么是数组？

NumPy 数组可以理解为**多维表格**：
- 一维数组（1D）：一列数字，比如 [1, 2, 3]
- 二维数组（2D）：行列表格，比如一个 Excel 工作表
- 三维数组（3D）：多张表格叠放在一起

与 Python 列表的区别：
- NumPy 数组中**所有元素类型相同**（比如都是小数）
- NumPy 的数学运算**极快**（底层是 C 语言实现）
- NumPy 数组有`形状`（shape）概念

In [ ]:
import numpy as np

# 创建一个简单的一维数组
一维数组 = np.array([1.0, 2.5, 3.7, 4.2])

print("一维数组:", 一维数组)
print("形状 (shape):", 一维数组.shape)   # (4,) 表示有4个元素
print("数据类型 (dtype):", 一维数组.dtype) # float64 = 双精度浮点数

### 用真实数据举例：水分子索引数组

在本项目中，每个水分子由**三个原子**组成：1个氧原子（O）+ 2个氢原子（H）。  
程序用一个形状为 `(n_water, 3)` 的二维数组来记录所有水分子：
- **行**：每行代表一个水分子
- **列**：[O的原子编号, H1的原子编号, H2的原子编号]

In [ ]:
import ase.io
from src.structure.utils import detect_water_molecule_indices
from src.structure.Analysis.WaterAnalysis._common import _parse_abc_from_md_inp

# 读取第一帧，并设置晶胞（详见第三章）
md_inp_path = DATA_DIR / "md.inp"
xyz_path    = DATA_DIR / "md-pos-1.xyz"

a_A, b_A, c_A = _parse_abc_from_md_inp(md_inp_path)
atoms = ase.io.read(str(xyz_path), index=0)  # 读取第一帧（index=0）
atoms.set_cell([a_A, b_A, c_A])
atoms.set_pbc([True, True, True])

# 检测水分子，得到 (n_water, 3) 数组
水分子索引 = detect_water_molecule_indices(atoms)

print(f"形状: {水分子索引.shape}")
print(f"  → {水分子索引.shape[0]} 个水分子，每行 3 列 [O_idx, H1_idx, H2_idx]")
print(f"数据类型: {水分子索引.dtype}")
print(f"\n前 5 个水分子的原子编号：")
print(水分子索引[:5])

### 可视化：用彩色网格展示水分子索引数组

把这个数组想象成一张表格：每行一个水分子，三列分别是 O、H1、H2 的编号。  
用颜色深浅表示编号大小，可以直观感受数组的结构。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# 只取前 30 个水分子来展示（太多的话图会很小）
展示数量 = 30
展示数组 = 水分子索引[:展示数量]

fig, ax = plt.subplots(figsize=(4, 8))
im = ax.imshow(展示数组, aspect='auto', cmap='viridis')

# 坐标轴标签
ax.set_xticks([0, 1, 2])
ax.set_xticklabels(['O_idx\n(氧原子编号)', 'H1_idx\n(氢原子1编号)', 'H2_idx\n(氢原子2编号)'], fontsize=9)
ax.set_ylabel('水分子编号（行索引）', fontsize=10)
ax.set_title(f'水分子索引数组\n（前 {展示数量} 个水分子）', fontsize=11)

# 在每个格子里标注数值
for i in range(展示数量):
    for j in range(3):
        ax.text(j, i, str(展示数组[i, j]), ha='center', va='center',
                fontsize=6, color='white' if 展示数组[i, j] > 150 else 'black')

plt.colorbar(im, ax=ax, label='原子编号')
plt.tight_layout()
plt.show()
print(f"数组形状: {展示数组.shape} → {展示数量} 行 × 3 列")

### 数组索引和切片

从数组中取出特定的部分，叫做**索引（indexing）**或**切片（slicing）**。  
规则：编号从 **0** 开始（不是1！）

In [ ]:
arr = 水分子索引  # 形状 (n_water, 3)

# 取第一个水分子（第0行）
print("第 0 个水分子（arr[0]）:", arr[0])
print("  O原子编号:", arr[0, 0])
print("  H1原子编号:", arr[0, 1])
print("  H2原子编号:", arr[0, 2])

print()

# 取所有水分子的 O 原子编号（第0列的全部行）
# 语法：arr[:, 列编号]，冒号表示"所有行"
所有O的编号 = arr[:, 0]
print(f"所有 O 原子编号（arr[:, 0]）: 形状={所有O的编号.shape}")
print("前10个:", 所有O的编号[:10])

print()

# 取前 5 个水分子
# 语法：arr[开始:结束]，包含开始，不包含结束
print("前 5 个水分子（arr[:5]）:")
print(arr[:5])

### 常用 NumPy 操作

NumPy 提供了很多方便的数学函数，直接对整个数组操作，不需要写循环。

In [ ]:
# 获取所有氧原子的 z 坐标（缩放坐标）
scaled = atoms.get_scaled_positions()  # 返回 (n_atoms, 3) 数组，值在 [0, 1]
O_编号 = arr[:, 0]                     # 所有氧原子的编号
O_z坐标_分数 = scaled[O_编号, 2]       # 取氧原子的 z 分数坐标

print("氧原子 z 坐标统计（分数坐标，范围 0~1）：")
print(f"  最小值: {np.min(O_z坐标_分数):.4f}")
print(f"  最大值: {np.max(O_z坐标_分数):.4f}")
print(f"  平均值: {np.mean(O_z坐标_分数):.4f}")
print(f"  标准差: {np.std(O_z坐标_分数):.4f}")

# np.histogram：把数据分成若干个区间，统计每个区间有多少数据
# 就像制作频率直方图
counts, bin_edges = np.histogram(O_z坐标_分数, bins=50)
print(f"\nnp.histogram 结果：")
print(f"  bins={len(counts)} 个区间")
print(f"  counts 形状: {counts.shape}")
print(f"  bin_edges 形状: {bin_edges.shape}  (比 counts 多一个，表示区间边界)")

---

## 2.2 Matplotlib 绘图

### 图（Figure）和轴（Axes）的概念

Matplotlib 用两个对象来管理绘图：
- **Figure（图）**：相当于整张画布或一张白纸
- **Axes（轴）**：画布上的一个坐标系（可以有多个坐标系在同一张画布上）

```
fig（画布）
└── ax（坐标系）
    ├── ax.plot(...)    绘制折线
    ├── ax.set_xlabel() 设置 x 轴标签
    └── ax.set_title()  设置标题
```

In [ ]:
# 用真实的密度数据画一条曲线
# 先从第一章生成的 CSV 文件中读取数据
import numpy as np

密度CSV路径 = OUTPUT_DIR / "water_mass_density_z_distribution_analysis.csv"

if 密度CSV路径.exists():
    # np.genfromtxt：从文本文件读取数组，skip_header=1 跳过标题行
    数据 = np.genfromtxt(密度CSV路径, delimiter=',', skip_header=1)
    距离_A = 数据[:, 1]    # 第2列：距界面距离（Å）
    密度_g_cm3 = 数据[:, 2] # 第3列：密度（g/cm³）
    
    print(f"读取了 {len(距离_A)} 个数据点")
    print(f"距离范围: {距离_A[0]:.2f} ~ {距离_A[-1]:.2f} Å")
    print(f"密度范围: {密度_g_cm3.min():.3f} ~ {密度_g_cm3.max():.3f} g/cm³")
else:
    print("未找到密度 CSV，请先运行第一章（01_quickstart.ipynb）生成输出文件")
    # 用模拟数据代替演示
    距离_A = np.linspace(0, 12, 120)
    密度_g_cm3 = 1.0 + 2.0 * np.exp(-距离_A) * np.cos(距离_A)

In [ ]:
# 绘制密度曲线
fig, ax = plt.subplots(figsize=(8, 4))  # 创建一个 8×4 英寸的画布

# 绘制折线
ax.plot(
    距离_A,          # x 轴数据
    密度_g_cm3,      # y 轴数据  
    color='tab:blue', # 颜色
    linewidth=1.5,    # 线宽
    label='水密度'    # 图例标签
)

# 添加标注线：体相水密度约 1.0 g/cm³
ax.axhline(y=1.0, color='gray', linestyle='--', linewidth=1, label='体相水参考值 (1.0 g/cm³)')

# 设置标签
ax.set_xlabel('距界面距离 (Å)', fontsize=12)  # x 轴标签
ax.set_ylabel('水密度 (g/cm³)', fontsize=12)  # y 轴标签
ax.set_title('水的质量密度分布（从界面到中心）', fontsize=13)
ax.legend(fontsize=10)
ax.set_xlim(0, None)  # x 轴从 0 开始

plt.tight_layout()  # 自动调整布局，防止标签被截断
plt.show()          # 在 notebook 中显示图片

### 常用绘图操作总结

| 操作 | 代码 |
|------|------|
| 折线图 | `ax.plot(x, y)` |
| 散点图 | `ax.scatter(x, y)` |
| 直方图 | `ax.hist(data, bins=50)` |
| 填充区域 | `ax.fill_between(x, y1, y2, alpha=0.3)` |
| 水平线 | `ax.axhline(y=1.0)` |
| 垂直线 | `ax.axvline(x=2.5)` |
| x 轴标签 | `ax.set_xlabel('距离 (Å)')` |
| 标题 | `ax.set_title('我的图')` |
| 图例 | `ax.legend()` |
| 保存图片 | `fig.savefig('output.png', dpi=150)` |

---

## 2.3 ASE Atoms 对象

### 什么是 Atoms 对象？

ASE（Atomic Simulation Environment）是一个处理原子结构的 Python 库。  
`Atoms` 对象就像一个**原子结构的容器**，里面存放了：
- 每个原子的**化学元素符号**（O, H, Cu, Ag...）
- 每个原子的**三维坐标**（x, y, z，单位 Å）
- **晶胞参数**（模拟盒子的形状和大小）
- **周期性边界条件**（PBC）设置

In [ ]:
import ase.io

# 读取第一帧（index=0）
atoms = ase.io.read(str(xyz_path), index=0)

# 设置晶胞（XYZ 文件不包含晶胞信息，需要从 md.inp 读取）
atoms.set_cell([a_A, b_A, c_A])
atoms.set_pbc([True, True, True])

print(f"Atoms 对象：")
print(f"  原子总数: {len(atoms)}")
print(f"  晶胞参数: a={a_A:.3f} Å, b={b_A:.3f} Å, c={c_A:.3f} Å")
print(f"  周期性边界: {atoms.pbc}")

In [ ]:
import pandas as pd

# 把 atoms 转成 pandas DataFrame，方便查看
# pandas DataFrame 就是 Python 中的"表格"对象
元素符号 = atoms.get_chemical_symbols()  # 列表：['Cu', 'Cu', ..., 'O', 'H', ...]
坐标_Angstrom = atoms.get_positions()    # (n_atoms, 3) 数组

df = pd.DataFrame({
    '元素': 元素符号,
    'x (Å)': 坐标_Angstrom[:, 0].round(3),
    'y (Å)': 坐标_Angstrom[:, 1].round(3),
    'z (Å)': 坐标_Angstrom[:, 2].round(3),
})

print(f"原子数据表格（共 {len(df)} 行）：")
print(f"\n前 5 个原子（金属）：")
print(df.head())
print(f"\n元素种类统计：")
print(df['元素'].value_counts())

In [ ]:
# 可视化：z坐标分布直方图，区分金属和水的原子
import numpy as np

元素数组 = np.array(元素符号)
z坐标 = 坐标_Angstrom[:, 2]

# 分类：金属原子（Cu, Ag）vs 水的原子（O, H）
金属掩码 = np.isin(元素数组, ['Cu', 'Ag'])
氧掩码   = (元素数组 == 'O')
氢掩码   = (元素数组 == 'H')

fig, ax = plt.subplots(figsize=(9, 4))

ax.hist(z坐标[金属掩码], bins=60, alpha=0.7, color='tab:orange', label=f'金属原子 Cu/Ag ({金属掩码.sum()} 个)')
ax.hist(z坐标[氧掩码],   bins=60, alpha=0.7, color='tab:red',    label=f'氧原子 O ({氧掩码.sum()} 个)')
ax.hist(z坐标[氢掩码],   bins=60, alpha=0.7, color='tab:blue',   label=f'氢原子 H ({氢掩码.sum()} 个)')

ax.set_xlabel('z 坐标 (Å)', fontsize=12)
ax.set_ylabel('原子数量', fontsize=12)
ax.set_title('第一帧各元素原子的 z 坐标分布', fontsize=13)
ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

print("\n解读：")
print("  - 金属原子聚集在 z 轴中间区域（金属板），形成几个尖峰（层状结构）")
print("  - 水分子（O 和 H）分布在金属板两侧（z < 低界面 和 z > 高界面）")

### 读取多帧轨迹

`index=':'` 是 Python 切片语法，表示"读取所有帧"。  
（这里只演示读取前 5 帧，完整读取需要几分钟）

In [ ]:
# 读取前 5 帧
前5帧 = ase.io.read(str(xyz_path), index=':5')  # index=':5' 表示第0到第4帧

print(f"读取了 {len(前5帧)} 帧")
print(f"每帧原子数: {[len(f) for f in 前5帧]}")

---

## 2.4 Python dataclass（数据类）

### 什么是 dataclass？

dataclass 是 Python 中一种特殊的类，专门用来**打包存储相关数据**。  
可以理解为**有名字的数据包**：不只是一个数字，而是一组有含义的数据放在一起。

类比：
- 普通元组 `(2.5, True, (0,0,1))` — 不知道各位置是什么含义
- dataclass `Layer(center_s=2.5, is_interface=True, normal_unit=(0,0,1))` — 每个字段都有明确名字

访问方式：`.字段名`

In [ ]:
from dataclasses import dataclass

# 一个简单的 dataclass 示例
@dataclass(frozen=True)  # frozen=True 表示创建后不可修改
class 水分子:
    O编号: int              # 氧原子在 atoms 中的编号
    H1编号: int             # 氢原子1的编号
    H2编号: int             # 氢原子2的编号
    z坐标_A: float         # 氧原子的 z 坐标（Å）
    在吸附层: bool = False   # 是否在吸附层中

# 创建一个实例
示例水分子 = 水分子(O编号=100, H1编号=101, H2编号=102, z坐标_A=3.5, 在吸附层=True)

# 访问字段
print("示例水分子：")
print(f"  O编号: {示例水分子.O编号}")
print(f"  H1编号: {示例水分子.H1编号}")
print(f"  z坐标: {示例水分子.z坐标_A} Å")
print(f"  是否在吸附层: {示例水分子.在吸附层}")

### 预告：本项目中的 dataclass

在后续章节中，你会遇到两个重要的 dataclass：

**`Layer`**（在 `LayerParser.py` 中）：
```python
@dataclass(frozen=True)
class Layer:
    atom_indices: tuple[int, ...]          # 该层所有金属原子的编号
    center_s: float                        # 层中心的 z 坐标（Å）
    is_interface: bool = False             # 是否是界面层
    normal_unit: tuple[float, ...] = None  # 法向量（指向水的方向）
```

**`SurfaceDetectionResult`**（在 `LayerParser.py` 中）：
```python
@dataclass(frozen=True)
class SurfaceDetectionResult:
    axis_unit: tuple[float, ...]           # 排序轴的单位向量
    metal_indices: tuple[int, ...]         # 所有金属原子的编号
    metal_layers_sorted: tuple[Layer, ...]  # 所有金属层（按 z 排序）
```

第四章会详细演示如何使用这两个 dataclass。

In [ ]:
# 快速预览：用真实数据创建 Layer dataclass
from src.structure.utils import Layer

示例层 = Layer(
    atom_indices=(0, 1, 2, 3, 4),  # 5个金属原子
    center_s=25.5,                  # 层中心在 z=25.5 Å
    is_interface=True,              # 这是界面层
    normal_unit=(0.0, 0.0, 1.0),   # 法向量朝 +z 方向（朝上，指向水）
)

print("Layer dataclass 示例：")
print(f"  原子编号: {示例层.atom_indices}")
print(f"  层中心位置: {示例层.center_s} Å")
print(f"  是界面层: {示例层.is_interface}")
print(f"  法向量（指向水方向）: {示例层.normal_unit}")
print(f"  法向量含义: z分量={示例层.normal_unit[2]:.0f} → 朝 +z 方向（朝上）")

---

## 本章小结

| 库 | 核心概念 | 在本项目中的用途 |
|----|---------|------------------|
| NumPy | 数组（ndarray）+ 形状（shape） | 存储原子坐标、密度曲线、索引数组 |
| Matplotlib | 图（Figure）+ 轴（Axes） | 绘制密度曲线、取向曲线、三联图 |
| ASE | Atoms 对象（原子结构容器） | 读取 XYZ 轨迹，获取原子坐标 |
| Python | dataclass（有名字的数据包） | Layer、SurfaceDetectionResult |

**下一章**（`03_input_data.ipynb`）将深入解析输入数据文件格式。